In [0]:
import pandas as pd

# EDGE_MAP {'element': {'start': , 'stop': 'num': }}

In [0]:

SAMPLE_MAP = {'sample1': {'name': 'AS-21_Spent', 'pos': 252, 'interesting_edges': []},
          'sample2': {'name': 'AS-21', 'pos': 259, 'interesting_edges': []},
          'sample3': {'name': 'AS-4-1_Spent', 'pos': 267, 'interesting_edges': []},
          'sample4': {'name': '30CoCeO2', 'pos': 276, 'interesting_edges': ['Ce_M', 'Co_L2', 'O_K']},
          'sample5': {'name': '8CoCeO2', 'pos': 282, 'interesting_edges': ['Ce_M2', 'Co_L', 'O_K']},
          'sample6': {'name': '2CoCeO2', 'pos': 290, 'interesting_edges': ['Ce_M', 'Co_L', 'O_K2']},
}


def mulit_sample_count(detectors, sample_map, position_motor,
                       md=None, **kwargs):
    dets = list(detectors) + [position_motor]
    for sample in sample_map.values():
        _md = dict(sample)
        _md.update(md or {})
        pos = sample['pos']
        yield from bp.mv(position_motor, pos)
        yield from bp.count(dets, md=_md, **kwargs)

In [0]:
pd.DataFrame().to_excel('asdf.xlsx')